In [ ]:
import pandas as pd
import numpy as np
from functools import reduce
import pyodbc
from sqlalchemy import create_engine
import urllib
from datetime import date
from pathlib import Path

In [ ]:
cnxn = pyodbc.connect(server = 'bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      database = 'ca_cannabis',
                      UID = 'admin',
                      PWD = 'N19lrqxnurTUJLJT6GFe')

In [ ]:
# Engine for Colorado Data
conxn = pyodbc.connect(server = 'bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      database = 'co_cannabis',
                      UID = 'admin',
                      PWD = 'N19lrqxnurTUJLJT6GFe')

In [ ]:
ca_roll = pd.read_sql("SELECT * FROM ca_roll", cnxn)
co_roll = pd.read_sql("SELECT * FROM co_roll", conxn)

df_roll = ca_roll.append(co_roll)
len(df_roll)

In [ ]:
df_roll.tail()

In [ ]:
filenames = Path('../ar_reports/orig/no_neg').glob('*.csv')
list_of_dfs = [pd.read_csv(file) for file in filenames]
for dataframe, file in zip(list_of_dfs, filenames):
    Dataframe['file'] = file
df_file = pd.concat(list_of_dfs, ignore_index=True, sort=False)

In [ ]:
df_file['date'] = pd.to_datetime(df_file['date'], format="%m/%d/%Y")

In [ ]:
df_ar = df_file

In [ ]:
df_ar['license_number'] = df_ar['license_number'].str.strip()

In [ ]:
df_ar_roll = pd.merge(df_ar, df_roll, on='license_number', how='left')
df_ar_roll.head()

In [ ]:
len(df_ar_roll[df_ar_roll['roll_up_id'].isnull()])

In [ ]:
df_ar_roll_fill = df_ar_roll[~df_ar_roll['roll_up_id'].isnull()]
len(df_ar_roll_fill)

In [ ]:
df_ar_roll_fill.head()

In [ ]:
df_ovr_10 = df_ar_roll_fill[df_ar_roll_fill['bal_out'] > 10]

In [ ]:
client_max_date = df_ovr_10[['client', 'date']].groupby('client', as_index=False).max()

In [ ]:
df_all = client_max_date.merge(df_ovr_10, on=['client', 'date'], how='left').sort_values(by='client', ascending=True)

In [ ]:
df_all[df_all['roll_up_id'] == '3510']

In [ ]:
df_ar_roll.to_csv(f'../../../combined_ar_list/combined_lists_with_null_co_ca_test.csv', index=False)
df_ar_roll_fill.to_csv(f'../../../combined_ar_list/combined_lists_co_ca_test.csv', index=False)

In [ ]:
df_ovr_10.to_csv(f'../../../combined_ar_list/underwriting_list_20210114.csv', index=False)